In [292]:
import pandas as pd
from datetime import date, timedelta
import urllib
import numpy as np
import altair as alt

In [293]:
# Global variables
today = date.today()
yesterday = date.today() - timedelta(days=1)

In [294]:
# Functions
def new_group(code, name, code_list, dataframe):
    df = dataframe[dataframe.code.isin(code_list)].groupby("date", as_index=False).sum()
    df["code"] = code
    df["name"] = name
    return df

def get_data(code,dataframe):
    return dataframe[(dataframe["code"] == code) & ((dataframe["date"] == pd.to_datetime(yesterday)) | (dataframe["date"] == pd.to_datetime(today)))]

In [295]:
# NHS
nhs_header_row = 15
nhs_trust_sheet_name = 'COVID19 total deaths by trust'
nhs_csv_trusts = 'nhs_csv_trusts.csv'
our_trust_codes = [
    'RYW', # Birmingham Community Healthcare NHS Foundation Trust
    'RNA', # The Dudley Group NHS Foundation Trust
    'RL4', # The Royal Wolverhampton NHS Trust
    'RXK', # Sandwell And West Birmingham Hospitals NHS Trust
    'RXW', # Shrewsbury And Telford Hospital NHS Trust
    'RRJ', # The Royal Orthopaedic Hospital NHS Foundation Trust
    'RRK', # University Hospitals Birmingham NHS Foundation Trust
    'RJE', # University Hospitals Of North Midlands NHS Trust
    'RBK', # Walsall Healthcare NHS Trust
    'RWP', # Worcestershire Acute Hospitals NHS Trust
    'R1A', # Worcestershire Health And Care NHS Trust
    'RL1', # The Robert Jones And Agnes Hunt Orthopaedic Hospital NHS Foundation Trust
    'RLY', # NORTH STAFFORDSHIRE COMBINED HEALTHCARE NHS TRUST
]
nhs_file_format = "https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/DIRECTORY/COVID-19-total-announced-deaths-DATE.xlsx"
nhs_file_date_format = "%-d-%B-%Y"
nhs_file_directory_format = "%Y/%m"
nhs_latest_file = False

nhs_today_file = nhs_file_format.replace("DATE",today.strftime(nhs_file_date_format))
nhs_today_file = nhs_today_file.replace("DIRECTORY",today.strftime(nhs_file_directory_format))

nhs_yesterday_file = nhs_file_format.replace("DATE",yesterday.strftime(nhs_file_date_format))
nhs_yesterday_file = nhs_yesterday_file.replace("DIRECTORY",yesterday.strftime(nhs_file_directory_format))

## NHS file handling with fallback
try:
  urllib.request.urlopen(nhs_today_file)
  nhs_latest_file = nhs_today_file
except urllib.error.HTTPError as e:
  try:
    nhs_latest_file = urllib.request.urlopen(nhs_yesterday_file)
    nhs_latest_file = nhs_yesterday_file
  except urllib.error.HTTPError as e:
    pass

## Import a specific sheet from the NHS Excel file and convert to CSV
nhs_excel = pd.read_excel(nhs_latest_file, header=nhs_header_row, sheet_name=nhs_trust_sheet_name)
nhs_excel.to_csv(nhs_csv_trusts)
nhs_trusts = pd.read_csv(nhs_csv_trusts)

## Filtering down to our_trusts
our_trusts = nhs_trusts[nhs_trusts.Code.isin(our_trust_codes)] # filter to our trusts
our_trusts = our_trusts.filter(regex='(^Code$|^Name$|[0-9]{4}-.+|^Up to.01-Mar-20$)', axis=1) # filter to only the required columns
our_trusts = our_trusts.rename(columns={"Code": "code", "Name" : "name"}) # clean up column names
our_trusts = pd.melt(our_trusts, id_vars=["code","name"], var_name="date", value_name="deaths") # un-pivot the date columns
our_trusts["date"] = our_trusts["date"].str.replace("Up to 01-Mar-20","2020-02-29 00:00:00")
our_trusts["date"] = pd.to_datetime(our_trusts.date) # clean all dates to YYYY-MM-DD

## Adding cumulative deaths
our_trusts["cumulative_deaths"] = our_trusts.groupby("name").cumsum()

## Added relative difference
our_trusts["reldiff"] = our_trusts.groupby("code")["cumulative_deaths"].pct_change().convert_dtypes(convert_integer=True) # add daily variance

## Cleaning float
our_trusts["deaths"] = our_trusts["deaths"].convert_dtypes(convert_integer=True)
our_trusts["cumulative_deaths"] = our_trusts["cumulative_deaths"].convert_dtypes(convert_integer=True)

## Additional Groups / SubTotals
### Create Groups
#### Total Region
group_totalregion_trusts = new_group("ManualREG","Total Region",[
    'RYW', # Birmingham Community Healthcare NHS Foundation Trust
    'RNA', # The Dudley Group NHS Foundation Trust
    'RL4', # The Royal Wolverhampton NHS Trust
    'RXK', # Sandwell And West Birmingham Hospitals NHS Trust
    'RRJ', # The Royal Orthopaedic Hospital NHS Foundation Trust
    'RRK', # University Hospitals Birmingham NHS Foundation Trust
    'RJE', # University Hospitals Of North Midlands NHS Trust
    'RBK', # Walsall Healthcare NHS Trust
    'RLY', # NORTH STAFFORDSHIRE COMBINED HEALTHCARE NHS TRUST
],our_trusts)
#### Black Country and Birmingham
group_blackcountryandbirmingham_trusts = new_group("ManualBAB","Black Country and Birmingham",[
    'RYW', # Birmingham Community Healthcare NHS Foundation Trust
    'RNA', # The Dudley Group NHS Foundation Trust
    'RL4', # The Royal Wolverhampton NHS Trust
    'RXK', # Sandwell And West Birmingham Hospitals NHS Trust
    'RRJ', # The Royal Orthopaedic Hospital NHS Foundation Trust
    'RRK', # University Hospitals Birmingham NHS Foundation Trust
    'RBK', # Walsall Healthcare NHS Trust
],our_trusts)
#### Sandwell and Birmingham
group_sandwellandbirmingham_trusts = new_group("ManualSAB","Sandwell and Birmingham",[
    'RYW', # Birmingham Community Healthcare NHS Foundation Trust
    'RXK', # Sandwell And West Birmingham Hospitals NHS Trust
    'RRJ', # The Royal Orthopaedic Hospital NHS Foundation Trust
    'RRK', # University Hospitals Birmingham NHS Foundation Trust
],our_trusts)

#### Worcestershire
group_worcestershire_trusts = new_group("ManualWOR","Worcestershire",[
    'RWP', # Worcestershire Acute Hospitals NHS Trust
    'R1A', # Worcestershire Health And Care NHS Trust
],our_trusts)

#### Staffordshire
group_staffordshire_trusts = new_group("ManualSTF","Staffordshire",[
    'RJE', # University Hospitals Of North Midlands NHS Trust
    'RLY', # NORTH STAFFORDSHIRE COMBINED HEALTHCARE NHS TRUST
],our_trusts)

### Add Groups
our_trust_groups = pd.DataFrame(columns = ["code","name","date","deaths","cumulative_deaths"])
our_trust_groups = our_trust_groups.append([
    group_sandwellandbirmingham_trusts,
    group_worcestershire_trusts,
    group_blackcountryandbirmingham_trusts,
    group_totalregion_trusts,
    group_staffordshire_trusts,
], ignore_index=True)

In [ ]:
# Public Health England (PHE)
our_utla_codes = [
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E06000051', # Shropshire
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
    'E06000020', # Telford and Wrekin
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
    'E10000034', # Worcestershire
]
pheh_dashboard_url = 'https://fingertips.phe.org.uk/documents/Historic%20COVID-19%20Dashboard%20Data.xlsx'

pheh_utla_header_row = 7 # zero indexed
pheh_utla_sheet = 'UTLAs'
pheh_utla_csv_filename = 'pheh-utla.csv'

pheh_ukdeaths_header_row = 7
pheh_ukdeaths_sheet = 'UK Deaths'
pheh_ukdeaths_csv_filename = 'pheh-ukdeaths.csv'

pheh_ukcases_header_row = 9 # changed around April 12
pheh_ukcases_sheet = 'Countries'
pheh_ukcases_csv_filename = 'pheh-ukcases.csv'

## Import the Countries / Cases sheet from the PHE Excel file and convert to CSV
pheh_ukcases_excel = pd.read_excel(pheh_dashboard_url, header=pheh_ukcases_header_row, sheet_name=pheh_ukcases_sheet)
pheh_ukcases_excel.to_csv(pheh_ukcases_csv_filename)
pheh_ukcases = pd.read_csv(pheh_ukcases_csv_filename)

### Refining the Countries / Cases data
pheh_ukcases = pheh_ukcases.rename(columns={
    "Area Code": "code",
    "Area Name": "name",
}) # clean up column names
pheh_ukcases.drop('Unnamed: 0', axis='columns', inplace=True)
pheh_ukcases = pd.melt(pheh_ukcases, id_vars=["code","name"], var_name="date", value_name="cases") # un-pivot the date columns
pheh_ukcases["date"] = pd.to_datetime(pheh_ukcases.date) # clean all dates to YYYY-MM-DD
pheh_ukcases["cases"] = pheh_ukcases["cases"].convert_dtypes(convert_integer=True)
pheh_ukcases.loc[pheh_ukcases.name == "UK", "code"] = "Manual UK"
pheh_ukcases["diff"] = pheh_ukcases.groupby("code")["cases"].diff().convert_dtypes(convert_integer=True) # add daily variance
pheh_ukcases["reldiff"] = pheh_ukcases.groupby("code")["cases"].pct_change().convert_dtypes(convert_integer=True) # add daily variance

## Import the UK Deaths sheet from the PHE Excel file and convert to CSV
pheh_ukdeaths_excel = pd.read_excel(pheh_dashboard_url, header=pheh_ukdeaths_header_row, sheet_name=pheh_ukdeaths_sheet)
pheh_ukdeaths_excel.to_csv(pheh_ukdeaths_csv_filename)
pheh_ukdeaths = pd.read_csv(pheh_ukdeaths_csv_filename)

### Refining the UK Deaths data
pheh_ukdeaths = pheh_ukdeaths.rename(columns={
    "Date": "date",
    "Deaths" : "deaths",
}) # clean up column names
pheh_ukdeaths.drop('Unnamed: 0', axis='columns', inplace=True)
pheh_ukdeaths.drop('deaths', axis='columns', inplace=True)
pheh_ukdeaths = pd.melt(pheh_ukdeaths, id_vars=["date"], var_name="name", value_name="deaths") # un-pivot the date columns
pheh_ukdeaths["date"] = pd.to_datetime(pheh_ukdeaths.date) # clean all dates to YYYY-MM-DD
pheh_ukdeaths["deaths"] = pheh_ukdeaths["deaths"].convert_dtypes(convert_integer=True)
# create a new column, setting a value of x where the name is y
country_code_conditions = [
    (pheh_ukdeaths["name"] == "England"),
    (pheh_ukdeaths["name"] == "Scotland"),
    (pheh_ukdeaths["name"] == "Wales"),
    (pheh_ukdeaths["name"] == "Northern Ireland"),
    (pheh_ukdeaths["name"] == "UK"),
]
country_code_choices = [
    "E92000001",
    "S92000003",
    "W92000004",
    "N92000002",
    "Manual UK",
]
pheh_ukdeaths["code"] = np.select(country_code_conditions,country_code_choices)
pheh_ukdeaths["diff"] = pheh_ukdeaths.groupby("code")["deaths"].diff().convert_dtypes(convert_integer=True) # add daily variance

## Import the UTLA sheet from the PHE Excel file and convert to CSV
pheh_utla_excel = pd.read_excel(pheh_dashboard_url, header=pheh_utla_header_row, sheet_name=pheh_utla_sheet)
pheh_utla_excel.to_csv(pheh_utla_csv_filename)
pheh_utla = pd.read_csv(pheh_utla_csv_filename)

### Filtering down to our UTLAs
our_utla = pheh_utla.rename(columns={"Area Code": "code", "Area Name" : "name"}) # clean up column names
our_utla = our_utla[our_utla.code.isin(our_utla_codes)] # filter to our trusts
our_utla.drop('Unnamed: 0', axis='columns', inplace=True)
our_utla = pd.melt(our_utla, id_vars=["code","name"], var_name="date", value_name="cases") # un-pivot the date columns
our_utla["date"] = pd.to_datetime(our_utla.date) # clean all dates to YYYY-MM-DD
our_utla["diff"] = our_utla.groupby("code")["cases"].diff().convert_dtypes(convert_integer=True) # add daily variance
our_utla["reldiff"] = our_utla.groupby("code")["cases"].pct_change().convert_dtypes(convert_integer=True) # add daily variance


## Additional Groups / SubTotals
### Create Groups
#### Total Region
group_totalregion_cases = new_group("ManualREG","Total Region",[
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
],our_utla)
#### Black Country
group_blackcountry_cases = new_group("ManualBLC","Black Country",[
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
],our_utla)
#### Black Country and Birmingham
group_blackcountryandbirmingham_cases = new_group("ManualBAB","Black Country and Birmingham",[
    'E08000025', # Birmingham
    'E08000027', # Dudley
    'E08000028', # Sandwell
    'E08000030', # Walsall
    'E08000031', # Wolverhampton
],our_utla)
#### Sandwell and Birmingham
group_sandwellandbirmingham_cases = new_group("ManualSAB","Sandwell and Birmingham",[
    'E08000025', # Birmingham
    'E08000028', # Sandwell
],our_utla)
#### Staffordshire
group_staffordshire_cases = new_group("ManualSTF","Staffordshire",[   
    'E10000028', # Staffordshire
    'E06000021', # Stoke-on-Trent
],our_utla)

### Add Groups
our_utla_groups = pd.DataFrame(columns = ["code","name","date","cases","diff"])
our_utla_groups = our_utla_groups.append([
    group_blackcountry_cases,
    group_sandwellandbirmingham_cases,
    group_staffordshire_cases,
    group_blackcountryandbirmingham_cases,
    group_totalregion_cases,
], ignore_index=True)

In [ ]:
## Data views
### Interesting
view_reductions = our_utla[our_utla["diff"] < 0] # instances where the cumulative case count went down
view_yesterday = our_utla[our_utla["date"] == pd.to_datetime(yesterday)]
view_today = our_utla[our_utla["date"] == pd.to_datetime(today)]
view_latest_deaths = our_trusts.pivot_table(our_trusts, index=["code","name"], aggfunc=np.sum)

### Required
#### Country
view_uk_cases = get_data("Manual UK",pheh_ukcases)
view_uk_deaths = get_data("Manual UK",pheh_ukdeaths)
view_england_cases = get_data("E92000001",pheh_ukcases)
view_england_deaths = get_data("E92000001",pheh_ukdeaths)
view_scotland_cases = get_data("S92000003",pheh_ukcases)
view_scotland_deaths = get_data("S92000003",pheh_ukdeaths)
view_wales_cases = get_data("W92000004",pheh_ukcases)
view_wales_deaths = get_data("W92000004",pheh_ukdeaths)
view_northernireland_cases = get_data("N92000002",pheh_ukcases)
view_northernireland_deaths = get_data("N92000002",pheh_ukdeaths)

#### Total Region
view_totalregion_cases = get_data("ManualREG",our_utla_groups)
view_totalregion_deaths = get_data("ManualREG",our_trust_groups)

#### Black Country and Birmingham
view_blackcountry_cases = get_data("ManualBLC",our_utla_groups)
view_blackcountryandbirmingham_cases = get_data("ManualBAB",our_utla_groups)
view_blackcountryandbirmingham_deaths = get_data("ManualBAB",our_trust_groups)

#### Wolverhampton
view_wolverhampton_cases = get_data("E08000031",our_utla)
view_wolverhampton_deaths = get_data("RL4",our_trusts)

#### Walsall
view_walsall_cases = get_data("E08000030",our_utla)
view_walsall_deaths = get_data("RBK",our_trusts)

#### Dudley
view_dudley_cases = get_data("E08000027",our_utla)
view_dudley_deaths = get_data("RNA",our_trusts)

#### Sandwell and Birmingham
view_sandwell_cases = get_data("E08000028",our_utla)
view_birmingham_cases = get_data("E08000025",our_utla)
view_total_sandwellandbirmingham_cases = get_data("ManualSAB",our_utla_groups)
view_total_sandwellandbirmingham_deaths = get_data("ManualSAB",our_trust_groups)

#### Staffordshire
view_staffordshire_cases = get_data("E10000028",our_utla)
view_stoke_cases = get_data("E06000021",our_utla)
view_total_staffordshire_cases = get_data("ManualSTF",our_utla_groups)
view_total_staffordshire_deaths = get_data("ManualSTF",our_trust_groups)

#### Worcestershire
view_worcestershire_cases = get_data("E10000034",our_utla)
view_worcetershire_deaths = get_data("ManualWOR",our_trust_groups)

# UK

In [ ]:
view_uk_cases

In [ ]:
view_uk_deaths

# England

In [ ]:
view_england_cases

In [ ]:
view_england_deaths

# Scotland

In [ ]:
view_scotland_cases

In [ ]:
view_scotland_deaths

# Wales

In [ ]:
view_wales_cases

In [ ]:
view_wales_deaths

# Northern Ireland

In [ ]:
view_northernireland_cases

In [ ]:
view_northernireland_deaths

# Total E&S Region
_Combining Black Country, Birmingham and Staffordshire._

In [ ]:
view_totalregion_cases

In [ ]:
view_totalregion_deaths

# Total Black Country and Birmingham

In [ ]:
view_blackcountry_cases

In [ ]:
view_blackcountryandbirmingham_cases

In [ ]:
view_blackcountryandbirmingham_deaths

# Wolverhampton

In [ ]:
view_wolverhampton_cases

In [ ]:
view_wolverhampton_deaths

# Walsall

In [ ]:
view_walsall_cases

In [ ]:
view_walsall_deaths

# Dudley

In [ ]:
view_dudley_cases

In [ ]:
view_dudley_deaths

# Sandwell and Birmingham

In [ ]:
view_sandwell_cases

In [ ]:
view_birmingham_cases

In [ ]:
view_total_sandwellandbirmingham_cases

In [ ]:
view_total_sandwellandbirmingham_deaths

# Staffordshire

In [ ]:
view_staffordshire_cases

In [ ]:
view_stoke_cases

In [ ]:
view_total_staffordshire_cases

## Total Staffordshire Deaths
_North Staffordshire Combined Healthcare NHS Trust (RLY) + University Hospitals of North Midlands NHS Trust (RJE)_

In [ ]:
view_total_staffordshire_deaths

# Worcestershire

In [ ]:
view_worcestershire_cases

In [ ]:
view_worcetershire_deaths

---
# Visualisations

## Cumulative Deaths at E&S region trusts

In [ ]:
eands_trust_codes = [
    'RNA', # The Dudley Group NHS Foundation Trust
    'RL4', # The Royal Wolverhampton NHS Trust
    'RXK', # Sandwell And West Birmingham Hospitals NHS Trust
    'RRK', # University Hospitals Birmingham NHS Foundation Trust
    'RBK', # Walsall Healthcare NHS Trust
    'RJE', # University Hospitals Of North Midlands NHS Trust
]
eands_trust_data = our_trusts[our_trusts.code.isin(eands_trust_codes)]

alt.Chart(eands_trust_data).mark_line().encode(
    x='date',
    y='cumulative_deaths',
    color='name',
    tooltip='name'
)

## Relative change in cases at country level

In [ ]:
alt.Chart(pheh_ukcases).mark_line().encode(
    x='date',
    y='reldiff',
    color='name',
    tooltip='name',
)

## Relative change in deaths at our trusts

In [ ]:
alt.Chart(our_trusts).mark_line().encode(
    x='date',
    y='reldiff',
    color='name',
    tooltip='name',
)

## Relative change in cases in our UTLAs

In [ ]:
alt.Chart(our_utla).mark_line().encode(
    x='date',
    y='reldiff',
    color='name',
    tooltip='name',
)